In [1]:
!pip install ortools
#!pip install dtale

     |████████████████████████████████| 32.7MB 117kB/s 


In [2]:
import pandas as pd
import numpy as np
import collections
from ortools.sat.python import cp_model
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 137 (delta 49), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (137/137), 2.13 MiB | 13.05 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/cloned-repo


In [3]:
#Importing Data
df=pd.read_excel('orders1.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

# Formating Data
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']
df['Order Qty'] = df['Order Qty']*1000
df=df[df['Key']!='17023287gd39000800iX']


# Creating Parameter Data
jobs =df['Key']
op =[1,2,3,4]  # Ignore as of now
mc = [1,2,3,4,5,6]
#horizon = max(df["due"])+900
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur['Output']=dur['Output']*100
dur['duration']=dur['Order Qty']/dur['Output']
dur['duration'] = dur['duration'].astype('float64').round(decimals = 2)
dur.set_index(['Key','Line'],inplace=True)
del dur['FG Code']
dur['Output'].replace(0,1,inplace=True)
df['Order Qty']=df['Order Qty'].fillna(0).astype('int')
dur['duration']=dur['duration'].fillna(0).astype('int')
dur['Output']=dur['Output'].fillna(0).astype('int')
horizon = max(dur['duration']*100)

In [4]:
#Testing
#dur['duration'].loc[('17023287gd29904300iX',1)]
#dur
setup = pd.DataFrame(columns=('Key1','Key2','setup'))
for i in df['Key']:
  for k in df['Key']:
    if df[df['Key']==i]["FG Code"].values[0]==df[df['Key']==k]["FG Code"].values[0]:
      f = {'Key1':[i],"Key2":[k],"setup":[0]}
    else:
      f = {'Key1':[i],"Key2":[k],"setup":[40]}
    f = pd.DataFrame(data=f)
    setup = pd.concat([setup,f])
setup.set_index(['Key1','Key2'],inplace=True)

In [ ]:
setup.loc[('17023287gd29904300iX','17023287gd29907800iX')].values[0]

40

In [ ]:
model = cp_model.CpModel()
task_type = collections.namedtuple('task_type', 'start end duration run1')
mtj = collections.defaultdict(list)
all_tasks = {}

for j in jobs:
  for m in mc:
      suffix = '_%s_%i' % (j,m)
      start = model.NewIntVar(0, horizon, "start"+suffix)
      end = model.NewIntVar(0, horizon, "end"+suffix)
      run1 = model.NewIntVar(0, dur['duration'].loc[(j,m)].item(), "run"+suffix)
      duration = model.NewIntervalVar(start, run1, end, "duration"+suffix)
      all_tasks[j, m] = task_type(start=start, end=end, duration=duration, run1=run1)
      mtj[m].append(duration)
 
for m in mc:
  model.AddNoOverlap(mtj[m])

com = {}
for j in jobs:
  tt= 0
  for m in mc:
    comp = all_tasks[j,m].run1*dur["Output"].loc[(j,m)]
    tt = tt + comp
  com[j] = tt
  model.Add(com[j]*10 >= df[df["Key"]==j]['Order Qty'].values[0])


a={}
for i in jobs:
  for k in jobs:
    for m in mc:
      a[i,k,m] = model.NewBoolVar('previous')
      
for i in jobs:
  for k in jobs:
    for m in mc:
      model.AddBoolOr([a[i,k,m],a[k,i,m]])
      model.Add(all_tasks[k, m].start - all_tasks[i, m].end >= setup.loc[(i,k)].values[0]).OnlyEnforceIf(a[i,k,m])
      model.Add(all_tasks[i, m].start - all_tasks[k, m].end >= setup.loc[(i,k)].values[0]).OnlyEnforceIf(a[i,k,m].Not())

'''
#com1 = {}
#for j in df['Key']:
#  tt= 0
#  for m in mc:
#    comp1 = (all_tasks[j,m].end - all_tasks[j,m].start)*dur["Output"].loc[(j,m)]
#    tt = tt + comp1
#  com1[j] = tt
#  model.Add(com1[j]*10 <= df[df["Key"]==j]['Order Qty'].values[0]+500)
'''

ck = model.NewIntVar(-horizon, horizon, 'dummy')
model.AddMaxEquality(ck,[all_tasks[j,m].end for j in jobs for m in mc])

model.Minimize(ck)
solver = cp_model.CpSolver()
solver.Solve(model)
solver.StatusName()
#solver.ObjectiveValue()

In [13]:
a

{('17023287gd29904300iX', '17023287gd29904300iX', 1): previous(0..1)}

In [ ]:
st={}
en={}
for m in mc:
  for j in jobs:
    st[j,m]=solver.Value(all_tasks[j,m].start)
    en[j,m]=solver.Value(all_tasks[j,m].end)
    #dur[j,m]=solver.Value(all_tasks[j,m].duration)
en

IndexError: ignored

In [ ]:
st

{('17023287gd29904300iX', 1): 0,
 ('17023287gd29904300iX', 2): 71,
 ('17023287gd29904300iX', 3): 102,
 ('17023287gd29904300iX', 4): 17,
 ('17023287gd29904300iX', 5): 18,
 ('17023287gd29904300iX', 6): 0,
 ('17023287gd29907800iX', 1): 5,
 ('17023287gd29907800iX', 2): 26,
 ('17023287gd29907800iX', 3): 102,
 ('17023287gd29907800iX', 4): 41,
 ('17023287gd29907800iX', 5): 9,
 ('17023287gd29907800iX', 6): 74,
 ('17023287gd55102400iX', 1): 12,
 ('17023287gd55102400iX', 2): 25,
 ('17023287gd55102400iX', 3): 2,
 ('17023287gd55102400iX', 4): 0,
 ('17023287gd55102400iX', 5): 0,
 ('17023287gd55102400iX', 6): 0,
 ('17023335gd28001700iX', 1): 22,
 ('17023335gd28001700iX', 2): 6,
 ('17023335gd28001700iX', 3): 3,
 ('17023335gd28001700iX', 4): 69,
 ('17023335gd28001700iX', 5): 18,
 ('17023335gd28001700iX', 6): 0,
 ('17023335gd28013000iX', 1): 0,
 ('17023335gd28013000iX', 2): 17,
 ('17023335gd28013000iX', 3): 2,
 ('17023335gd28013000iX', 4): 102,
 ('17023335gd28013000iX', 5): 47,
 ('17023335gd28013000iX'

In [ ]:
#Exploring Data
 
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
show(df)

https://n6ekujl3anh-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [ ]:
model = cp_model.CpModel()
task_type = collections.namedtuple('task_type', 'start end duration')
mtj = collections.defaultdict(list)
all_tasks = {}
for m in mc:
  for j in jobs:  
      suffix = '_%s_%i' % (j,m)
      start = model.NewIntVar(0, horizon, "start"+suffix)
      end = model.NewIntVar(0, horizon, "end"+suffix)
      duration = model.NewIntervalVar(start, dur['duration'].loc[(j,m)], end, "duration"+suffix)
      all_tasks[(j, m)] = task_type(start=start, end=end, duration=duration)
      mtj[m].append(duration)

for m in mc:
    #mjob=[]
    #mtj[m] = mjob
    model.AddNoOverlap(mtj[m])

com = {}
for j in df['Key']:
  tt= 0
  for m in mc:
    #model.Add((all_tasks[(j, m)].end-all_tasks[(j, m)].start)*dur['Output'].loc[(j,m)] >= df[df["Key"]==j]['Order Qty'].values[0])
    comp = (all_tasks[(j,m)].end - all_tasks[(j,m)].start)*dur["Output"].loc[(j,m)]
    tt = tt + comp
  com[j] = tt
  model.Add(com[j] >= df[df["Key"]==j]['Order Qty'].values[0])
    # Remove 0 from packing output, it is causing error

com1 = {}
for j in df['Key']:
  tt= 0
  for m in mc:
    #model.Add((all_tasks[(j, m)].end-all_tasks[(j, m)].start)*dur['Output'].loc[(j,m)] >= df[df["Key"]==j]['Order Qty'].values[0])
    comp1 = (all_tasks[(j,m)].end - all_tasks[(j,m)].start)*dur["Output"].loc[(j,m)]
    tt = tt + comp1
  com1[j] = tt
  model.Add(com1[j] <= df[df["Key"]==j]['Order Qty'].values[0]+300)

delay = 0
for j in df['Key']:
  dd=[]
  for m in mc:
    dd.append(due['due'].loc[j]-all_tasks[(j, m)].end)
  delay += max(dd)

dummy = model.NewIntVar(-horizon, horizon, 'dummy')
#dummy=0
model.Add(delay<=dummy)
model.Add(-delay<=dummy)

In [ ]:
model.Minimize(dummy)
solver = cp_model.CpSolver()
solver.Solve(model)
solver.StatusName()

'INFEASIBLE'

In [ ]:
delay

SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(S

**Indices**  
j - job  
m - machine  

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  

**Objective**  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$  

**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $
